![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
# Desafio 1:

### Grupo 6:
- Ruiz Martínez,Santiago
- Fairhurst, Guillermo
- Castellari, Diego
- Ripoll, Rodrigo



![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)
## Importamos las bibliotecas que vamos a utilizar:

In [1]:
import os
import numpy as np
import pandas as pd
import re
import math
import seaborn as sns
import matplotlib.pyplot as plt

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)
## Step 1: Importamos el Dataframe de Properatti y realizamos primeras observaciones:

In [2]:
df = pd.read_csv('properatti.csv')

In [3]:
pd.set_option("display.max_columns", None)  # para poder ver todas las columnas
df.head(3)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,62000.0,USD,1093959.0,62000.0,55.0,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,150000.0,USD,2646675.0,150000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982,72000.0,USD,1270404.0,72000.0,55.0,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...


In [4]:
print('Columnas:                     Count:          dtype:')
print('-'*55)

df.info()

Columnas:                     Count:          dtype:
-------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121220 entries, 0 to 121219
Data columns (total 26 columns):
Unnamed: 0                    121220 non-null int64
operation                     121220 non-null object
property_type                 121220 non-null object
place_name                    121197 non-null object
place_with_parent_names       121220 non-null object
country_name                  121220 non-null object
state_name                    121220 non-null object
geonames_id                   102503 non-null float64
lat-lon                       69670 non-null object
lat                           69670 non-null float64
lon                           69670 non-null float64
price                         100810 non-null float64
currency                      100809 non-null object
price_aprox_local_currency    100810 non-null float64
price_aprox_usd               100810 

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)
## Step 2: health check

Realizamos un primer análisis exploratorio del dataset, buscamos ver que informacion es relevante y detectamos información faltante que pueda llegar a ser completada utilizando otros campos

__**Primeras observaciones:**__  
1) Para el presente ejercicio, las siguientes columnas pueden ser eliminadas:  
  - `Unnamed: 0`: repite la informacion del indice autogenerado por pandas   
  - `country`: se puede eliminar porque todas las propiedades son de Argentina.  
  - `geonames_id`: no encontramos relevancia sobre la columna ya que la informacion geografica faltante puede obtenerse de la columna `place_name`  
  - `lat-lon`: consolida a las columnas `lat` y `lon`. Consideramos de mayor utilidad dejar la información de latitud y longitud por separado por lo que se puede eliminar y asi evitar redundancia. Los datos faltantes de `lat` y `lon` pueden obtenerse de la columna `place_name` como se mencionó anteriormente.  
  - `properati_url`: carece de utilidad para el presente ejercicio  
  - `image_thumbnail` carece de utilidad para el presente ejercicio  
  
2) las columnas `operation`, `property_type` y `currency` puede cambiarse el tipo de dato a "categorical" para optimizar el uso de memoria.  
3) la columna `rooms` puede ser completada aplicando regex sobre las columnas `description` y `title`.  
4) `place_with_parent_names` agrupa a las columnas `country`, `state_name` y `place_name` de esta primera se pueden completar los 23 campos faltantes de `place_name` que casualmente son "Tigre".   
5) trabajamos con las columnas de precios.  
6) detectamos outliers.  
7) Observamos que todas las columnas con datos quantitativos estan en formato float, por lo que no requieren acciones adicionales. 


.

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)
### 2.1 Limpieza del Dataset 

In [5]:
# guardamos 1 copia del original
df_orig = df.copy()

#### 2.1.1 Eliminamos las columnas innecesarias:

In [6]:
# la columna 'Unnamed: 0'es un indice es innecesaria, la eliminamos
df = df.drop(['Unnamed: 0','country_name','geonames_id','lat-lon','properati_url','image_thumbnail'], axis=1)

#### 2.1.2 Cambiar el tipo de datos a categorico cuando corresposonde.  
La columna `operations` posee un unico valor: sell. Como la hoja de consigna indica que potencialmente puede tomar 2 posibles 
valores (sell/rent) no eliminamos la columna, pero la hacemos del tipo category.  
Hacemos lo mismo con las columnas `property_type` y `currency`.

In [7]:
df['operation'] = df['operation'].astype('category')
df['property_type'] = df['property_type'].astype('category')
df['currency'] = df['currency'].astype('category')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121220 entries, 0 to 121219
Data columns (total 20 columns):
operation                     121220 non-null category
property_type                 121220 non-null category
place_name                    121197 non-null object
place_with_parent_names       121220 non-null object
state_name                    121220 non-null object
lat                           69670 non-null float64
lon                           69670 non-null float64
price                         100810 non-null float64
currency                      100809 non-null category
price_aprox_local_currency    100810 non-null float64
price_aprox_usd               100810 non-null float64
surface_total_in_m2           81892 non-null float64
surface_covered_in_m2         101313 non-null float64
price_usd_per_m2              68617 non-null float64
price_per_m2                  87658 non-null float64
floor                         7899 non-null float64
rooms                         47

__**nota:**__ se redujo el espacio en memorio del dataset en 8MB (33%) lo cual hara mas eficiente todas las operaciones a continuación 

#### 2.1.3 inferir valores faltantes del feature 'room' 
A continuación vamos a completar la informacion faltante de la columna `rooms` con la informacion que pueda obtenerse de las columnas `description` y `title`.  
Para ello vamos a utilizar la libreria ***re***

Definmos una funcion `cant_amb()` que utilizaremos para obtener la cantidad de ambientes mencionadas en la columnas de `description` y `title`.

In [8]:
def cant_amb(text):
    ambRegex = re.compile(r'''
        (mono)     #intento matchear monoambiente
        \s?
        amb.?\w*
        |          # o
        (\d)       # cantidad de ambientes
        \s?        # espacio - puede estar o no -
        amb.?\w*   # segido por amb, amb. o ambiente
        ''', re.VERBOSE | re.IGNORECASE)
    try:
        mo = ambRegex.search(text)
        #print(mo.group(1))
        if mo.group(1) != None:
            return 1
        if mo.group(2) != None:
            return int(mo.group(2))
    except:
        return np.nan

In [9]:
# extraemos la cantidad de ambientes mencionados en las columnas descriptivas.
amb = df[['description','title']].applymap(lambda x: cant_amb(x))
amb.head()

,description,title
0,2.0,2.0
1,NaN,NaN
2,2.0,2.0
3,3.0,3.0
4,NaN,2.0


In [10]:
amb = pd.concat([amb, df['rooms']], axis=1)

#consolidamos la informacion extraida de las columnas descriptivas en una nueva variable amb_unif
amb['unif'] = amb.apply(lambda x: math.ceil(x.sum()/x.count()) if x.count() != 0 else np.nan,axis=1)

In [11]:
#consolidamos la informacion extraida de las columnas descriptivas en una nueva variable amb_unif
amb_unif = amb.apply(lambda x: x.sum()/x.count() if x.count() != 0 else np.nan,axis=1)

In [12]:
amb['unif'].count()

73291

Vemos que previamente a este ejercicio teníamos 52,004 campos con valores para la columna `room` y luego del ejercicio tenemos 73,291 (logramos recuperar un 40% de datos adicionales a los dators originales sobre la columna room. 

In [13]:
df['room']= amb['unif']

.

#### 2.1.4 Trabajamos sobre los features `place_with_parent_names` , `country`, `state_name` y `place_name`  
`place_with_parent_names` agrupa a las columnas `country`, `state_name` y `place_name` de esta primera se pueden completar los 23 campos faltantes de `place_name` que casualmente son "Tigre".  

Ahora analisamos los 23 campos de la columna `place_name` que estan vacios. 
Se observa que todos los valores deben ser completados con *"Tigre"*

In [14]:
mask = df['place_name'].isnull()
display(df[mask])


,operation,property_type,place_name,place_with_parent_names,state_name,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,description,title,room
6489,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Bs.As. G.B.A. Zona Norte,-34.400897,-58.638098,650000.0,USD,11468925.0,650000.0,240.0,240.0,2708.333333,2708.333333,NaN,NaN,NaN,"Espectacular casa moderna de una planta, con a...",Venta de casa en Santa María de Tigre,NaN
10201,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Bs.As. G.B.A. Zona Norte,-34.400704,-58.638067,535000.0,USD,9439807.5,535000.0,NaN,300.0,NaN,1783.333333,NaN,5.0,NaN,Corredor Responsable: SABRINA COCCONI - CSI 62...,VENTA CASA SANTA MARÍA DE TIGRE COUNTRY BARRIO...,5.0
11451,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Bs.As. G.B.A. Zona Norte,-34.402688,-58.626261,550000.0,USD,9704475.0,550000.0,2170.0,270.0,253.456221,2037.037037,NaN,NaN,NaN,MUY LINDA CASA EN EXCELENTE UBICACION!! Superf...,SANTA MARIA DE TIGRE IMPECABLE,NaN
14839,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Bs.As. G.B.A. Zona Norte,-34.400799,-58.638174,480000.0,USD,8469360.0,480000.0,240.0,200.0,2000.000000,2400.000000,NaN,NaN,NaN,Muy linda casa de estilo clásico.-Planta baja;...,Venta de casa en Santa María de Tigre,NaN
18622,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Bs.As. G.B.A. Zona Norte,-34.403278,-58.626350,1297000.0,USD,22884916.5,1297000.0,340.0,340.0,3814.705882,3814.705882,NaN,NaN,NaN,Casa con increible vista al lago con playa de...,Casa con increible vista al lago con playa y m...,NaN
21922,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Bs.As. G.B.A. Zona Norte,-34.399704,-58.638405,440000.0,USD,7763580.0,440000.0,NaN,220.0,NaN,2000.000000,NaN,5.0,NaN,Corredor Responsable: Matias Castelli - CUCICB...,Venta Casa - Barrio Santa Maria de Tigre,5.0
23664,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Bs.As. G.B.A. Zona Norte,-34.400516,-58.641800,480000.0,USD,8469360.0,480000.0,294.0,294.0,1632.653061,1632.653061,NaN,NaN,NaN,EXCELENTE CASA CLASICA con grandes ventanas y ...,CASA EN VENTA EN SANTA MARIA DE TIGRE CON RENTA,NaN
24722,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Bs.As. G.B.A. Zona Norte,-34.400729,-58.637778,530000.0,USD,9351585.0,530000.0,316.0,270.0,1677.215190,1962.962963,NaN,NaN,NaN,"EN EXCLUSIVIDAD!!!!!!Lindísima casa, muy lumin...",Venta de casa en Santa María de Tigre,NaN
38856,sell,house,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Bs.As. G.B.A. Zona Norte,-34.425087,-58.579659,1350000.0,USD,23820075.0,1350000.0,NaN,550.0,NaN,2454.545455,NaN,NaN,NaN,SANTA MARIA DE TIGRE- AL RIO - ESPECTACULAR CA...,"Casa (barrio cerrado), Venta",NaN
45970,sell,apartment,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Bs.As. G.B.A. Zona Norte,-34.396074,-58.637380,95000.0,USD,1676227.5,95000.0,55.0,27.0,1727.272727,3518.518519,NaN,1.0,NaN,"EDIFICIO ZERENA – EL PALMAR, NORDELTA. Torre 2...",Excelente monoambiente. Barrio El Palmar. Con...,2.0


In [15]:
df.loc[df['place_name'].isnull(),'place_name']='Tigre'
display(df[mask].head(3))

,operation,property_type,place_name,place_with_parent_names,state_name,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,description,title,room
6489,sell,house,Tigre,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Bs.As. G.B.A. Zona Norte,-34.400897,-58.638098,650000.0,USD,11468925.0,650000.0,240.0,240.0,2708.333333,2708.333333,NaN,NaN,NaN,"Espectacular casa moderna de una planta, con a...",Venta de casa en Santa María de Tigre,NaN
10201,sell,house,Tigre,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Bs.As. G.B.A. Zona Norte,-34.400704,-58.638067,535000.0,USD,9439807.5,535000.0,NaN,300.0,NaN,1783.333333,NaN,5.0,NaN,Corredor Responsable: SABRINA COCCONI - CSI 62...,VENTA CASA SANTA MARÍA DE TIGRE COUNTRY BARRIO...,5.0
11451,sell,house,Tigre,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||,Bs.As. G.B.A. Zona Norte,-34.402688,-58.626261,550000.0,USD,9704475.0,550000.0,2170.0,270.0,253.456221,2037.037037,NaN,NaN,NaN,MUY LINDA CASA EN EXCELENTE UBICACION!! Superf...,SANTA MARIA DE TIGRE IMPECABLE,NaN


__**nota:**__ verificamos que los valores vacios de la columna `place_name` fueron completados correctamente.

Adicionalmente Observamos que la descripcion dentro de `place_with_parent_names` tiene hasta 4 niveles de detalle.  
Manipulamos la columna para quedarnos unicamente con los primeros 3 niveles.
También buscamos uniformizar los nombres de la provincia de Buenos Aires.

In [19]:
def geo_limit(text):
    address_list = text.split('|')
    pais = address_list[1]
    provincia = address_list[2]
    barrio = address_list[3]
    
    if len(barrio)==0: barrio = provincia 
    if 'bs.as' in provincia.lower(): provincia = 'Buenos Aires'
    if 'buenos aires' in provincia.lower(): provincia = 'Buenos Aires'
        

    return barrio.title()+", "+provincia.title()+", "+pais.title()    

In [20]:
#probamos la función
geo_limit('|Argentina|buenos aires|palermo|caca|')

'Palermo, Buenos Aires, Argentina'

In [21]:
df['place_with_parent_names'] = df['place_with_parent_names'].apply(geo_limit)

In [22]:
df

,operation,property_type,place_name,place_with_parent_names,state_name,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,description,title,room
0,sell,PH,Mataderos,"Mataderos, Capital Federal, Argentina",Capital Federal,-34.661824,-58.508839,62000.0,USD,1093959.00,62000.00,55.0,40.0,1127.272727,1550.000000,NaN,NaN,NaN,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,2.0
1,sell,apartment,La Plata,"La Plata, Buenos Aires, Argentina",Bs.As. G.B.A. Zona Sur,-34.903883,-57.964330,150000.0,USD,2646675.00,150000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,NaN
2,sell,apartment,Mataderos,"Mataderos, Capital Federal, Argentina",Capital Federal,-34.652262,-58.522982,72000.0,USD,1270404.00,72000.00,55.0,55.0,1309.090909,1309.090909,NaN,NaN,NaN,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,2.0
3,sell,PH,Liniers,"Liniers, Capital Federal, Argentina",Capital Federal,-34.647797,-58.516424,95000.0,USD,1676227.50,95000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,3.0
4,sell,apartment,Centro,"Mar Del Plata, Buenos Aires, Argentina",Buenos Aires Costa Atlántica,-38.002626,-57.549447,64000.0,USD,1129248.00,64000.00,35.0,35.0,1828.571429,1828.571429,NaN,NaN,NaN,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,2.0
5,sell,house,Gualeguaychú,"Gualeguaychú, Entre Ríos, Argentina",Entre Ríos,-33.014071,-58.519828,NaN,NaN,NaN,NaN,53.0,NaN,NaN,NaN,NaN,NaN,NaN,"Casa en el perímetro del barrio 338, ubicada e...","Casa Barrio 338. Sobre calle 3 de caballería, ...",NaN
6,sell,PH,Munro,"Vicente López, Buenos Aires, Argentina",Bs.As. G.B.A. Zona Norte,-34.532957,-58.521782,130000.0,USD,2293785.00,130000.00,106.0,78.0,1226.415094,1666.666667,NaN,NaN,NaN,MUY BUEN PH AL FRENTE CON ENTRADA INDEPENDIENT...,"MUY BUEN PH AL FRENTE DOS DORMITORIOS , PATIO,...",NaN
7,sell,apartment,Belgrano,"Belgrano, Capital Federal, Argentina",Capital Federal,-34.559873,-58.443362,138000.0,USD,2434941.00,138000.00,45.0,40.0,3066.666667,3450.000000,NaN,NaN,NaN,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...,JOSE HERNANDEZ 1400 MONOAMBIENTE ESTRENAR CAT...,1.0
8,sell,apartment,Belgrano,"Belgrano, Capital Federal, Argentina",Capital Federal,-34.559873,-58.443362,195000.0,USD,3440677.50,195000.00,65.0,60.0,3000.000000,3250.000000,NaN,NaN,NaN,EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER...,"JOSE HERNANDEZ 1400 DOS AMBIENTES ESTRENAR ,...",NaN
9,sell,house,Rosario,"Rosario, Santa Fe, Argentina",Santa Fe,-32.942031,-60.725919,460000.0,ARS,455201.45,25798.49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MEDNOZA AL 7600A UNA CUADRA DE CALLE MENDOZAWH...,WHITE 7637 - 2 DORMITORIOS CON PATIO,NaN


A continuacion creamos una lista con todos los nombres de los barrios, que será utilizada en un futuro para:  
1) a fin de poder capturar sus correspondientes ubicaciones geograficas (lat y lon) y completar en las ocurrencias del dataset que faltasen.  
2) calcular los precios promedio del m2 por barrio.

In [23]:
lista_de_barrios = list(df['place_with_parent_names'].unique())
len(lista_de_barrios)

575

En este punto consideramos que no es necesario conservar la información de las columnas `state_name` y `place_name` ya que de ser necesarias a futuro se pueden recuperar de la columna `place_with_parent_names`

In [24]:
df = df.drop(['state_name','place_name'], axis=1)

.

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

#### 2.1.5 Trabajamos con las columnas de Precios para detectar inconsistencias y completar valores que puedan ser deducidos

Eliminamos las filas que no posen precio ni suficientes datos para inferirlos (ej. no posen precio x metro cuadrado ni superficie)

In [25]:
mask = df['price'].isnull() & df['currency'].isnull() & df['price_aprox_local_currency'].isnull() & df['price_aprox_usd'].isnull() & df['surface_total_in_m2'].isnull() & df['surface_covered_in_m2'].isnull() & df['price_usd_per_m2'].isnull() & df['price_per_m2'].isnull()
print('cantidad de filas antes de eliminar:, ', df['operation'].count())
df.drop(df[mask].index, inplace=True)
print('cantidad de filas despues de eliminar:, ', df['operation'].count())

cantidad de filas antes de eliminar:,  121220
cantidad de filas despues de eliminar:,  116487


creamos una serie con multi-indice para calcular el precio promedio del m2 por barrio, que luego utilizaremos para completar datos faltantes de la columna de precio por m2

In [26]:
ds_precio_prom_barrio = df.groupby(['property_type','place_with_parent_names'])['price_usd_per_m2'].mean()
ds_precio_prom_barrio

property_type  place_with_parent_names                              
PH             Abasto, Capital Federal, Argentina                       1570.485350
               Agronomía, Capital Federal, Argentina                    1566.441441
               Aguas Verdes, Buenos Aires, Argentina                     545.454545
               Almagro, Capital Federal, Argentina                      1760.360408
               Almirante Brown, Buenos Aires, Argentina                 1386.907236
               Avellaneda, Buenos Aires, Argentina                      1116.759941
               Bahía Blanca, Buenos Aires, Argentina                    1206.016095
               Balcarce, Buenos Aires, Argentina                         412.500000
               Balvanera, Capital Federal, Argentina                    1382.313309
               Barracas, Capital Federal, Argentina                     1233.943049
               Barrio Norte, Capital Federal, Argentina                 2577.079487
       

Creamos una funcion `price_usd_m2_fillna` para completar los campos vacios de precio_m2_usd que puedan ser inferidos como el promedio del barrio.  
Para ello utilizamos una mascara para filtrar aquellas filas que tienen un valor para el barrio, pero no tienen un valor para el precio_m2_usd.

In [27]:
mask_precio_m2_null = df['place_with_parent_names'].notnull() & df['price_usd_per_m2'].isnull()

def price_usd_m2_fillna(serie_fila,serie_referencia):
    prop_type = serie_fila['property_type']
    address = serie_fila['place_with_parent_names']
    #print(prop_type)
    #print(address)
    precio_usd_m2_x_proptype_prom = serie_referencia[(prop_type,address)]
    return precio_usd_m2_x_proptype_prom

#testeamos la función.
price_usd_m2_fillna(df.iloc[17,:], ds_precio_prom_barrio)
    

2403.9641183165736

In [29]:
df.loc[mask_precio_m2_null,'price_usd_per_m2'] = df[mask_precio_m2_null].apply(lambda fila: price_usd_m2_fillna(fila, ds_precio_prom_barrio), axis=1)
df[mask_precio_m2_null].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47870 entries, 1 to 121214
Data columns (total 19 columns):
operation                     47870 non-null category
property_type                 47870 non-null category
place_with_parent_names       47870 non-null object
lat                           26139 non-null float64
lon                           26139 non-null float64
price                         32193 non-null float64
currency                      32192 non-null category
price_aprox_local_currency    32193 non-null float64
price_aprox_usd               32193 non-null float64
surface_total_in_m2           13275 non-null float64
surface_covered_in_m2         37961 non-null float64
price_usd_per_m2              46934 non-null float64
price_per_m2                  24308 non-null float64
floor                         4298 non-null float64
rooms                         19842 non-null float64
expenses                      3195 non-null float64
description                   47868 non-nu

Volvemos a tratar de recuperar datos, esta vez obtenemos `price_usd_per_m2` a partir de `price_aprox_usd` y `surface_covered_in_m2`  

In [ ]:
mask_precio_m2_null = df['price_aprox_usd'].notnull() & df['surface_covered_in_m2'].notnull() & df['price_usd_per_m2'].isnull()

In [ ]:
df.loc[mask_precio_m2_null,'price_usd_per_m2'] = df.loc[mask_precio_m2_null].apply(lambda fila: fila['price_aprox_usd']/fila['surface_covered_in_m2'], axis=1)

ahora buscamos completar los `price_aprox_usd` a partir de `price_usd_per_m2`

In [ ]:
mask_precio_usd_null = df['price_aprox_usd'].isnull() & df['price_usd_per_m2'].notnull() & df['surface_covered_in_m2'].notnull()
df.loc[mask_precio_usd_null,'price_usd_per_m2'] = df[mask_precio_usd_null].apply(lambda fila: fila['surface_covered_in_m2']*fila['price_usd_per_m2'], axis=1)
df.info()

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)


### Análisis precio Venta

In [ ]:
df_ph = df_corr[df_corr['property_type']=='PH'].copy()
df_apartment = df_corr[df_corr['property_type']=='apartment'].copy()
df_house = df_corr[df_corr['property_type']=='house'].copy()
df_store = df_corr[df_corr['property_type']=='store'].copy()

In [ ]:
sns.pairplot(df_house,hue='place_with_parent_names')

In [ ]:
df_corr.corr()

In [ ]:
corr = df_corr.corr()
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

fig, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

Una vez que identificamos la correlacion entre variables, podemos ver de que manera estan correlacionadas. Por ejemplo: `price` and `price_per_m2`, `price_aprox_local_currency` and `price_aprox_usd`.

In [ ]:
df.plot(x='price_per_m2', y='price', kind='scatter', figsize=(12, 7))

In [ ]:
df.plot(x='price_aprox_local_currency', y='price', kind='scatter', figsize=(12, 7))

In [ ]:
df.plot(x='price_aprox_usd', y='price', kind='scatter', figsize=(12, 7))

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

__**A continuación vamos a completar la informacion de las columnas `lat` y `lon` con la informacion de `place_name`.**__  
Para ello vamos a utilizar la librerio ***geopy***

1) El primer paso es modificar el formato de la columna `place_with_parent_names` para poder capturar la lat y lon segun el barrio.

In [ ]:
df.loc[:,'place_with_parent_names'] = df.loc[:,'place_with_parent_names'].str.replace('|',',')
df.head()

2) a continuación instalamos la librería ***geopy*** y generamos 3 columnas nuevas:  
- `geopy`  
- `geopy_lat`  
- `geopy_lon`

In [ ]:
#!pip install geopy

In [ ]:
from geopy.geocoders import Nominatim  
nom = Nominatim(user_agent = "my-application")

# definimos una funcion que incorporte el try: / except:
def my_geocode(location):
    print('.',end='')
    try:
        return nom.geocode(location)
    except:
        return None

# probamos que la libreria este funcionando correctamente
display(nom.geocode(',Argentina,Capital Federal,Mataderos,'))
display(my_geocode(',Argentina,Capital Federal,Mataderos,'))

In [ ]:
df.loc[df['lat'].isnull(),'geopy'] = df.loc[df['lat'].isnull(),'place_with_parent_names'].apply(my_geocode)

Obtener la latitud y longitud a partir del nombre del la ubicacion, es un proceso que demora un tiempo considerable corriendo (~4hs). Es por esto que en este punto vamos a guardar el dataset

In [ ]:
df['geopy_lat'] = df['geopy'].apply(lambda x: x.latitude)
df['geopy_lon'] = df['geopy'].apply(lambda x: x.longitude)

In [ ]:
x.

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)
### 2.2 Detección de outliers

In [ ]:
df.describe()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 7))
sns.boxplot(data=df[['price_aprox_usd', 'surface_covered_in_m2']], orient="h", palette="Set2")

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)
### 2.3 Análisis de correlación:

Para el análisis de correlación trabajamos con los tipos de propiedad por separado.  
Adicionalmente eliminamos las columnas de latitud y longitud, etc

In [ ]:
df_corr = df[['property_type', 'place_name', 'place_with_parent_names', 'state_name', 
              'currency', 'price_aprox_usd', 'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'rooms']]

Eliminamos aquellas filas que no tienen precio para que no distorcionen la correlacion.

In [ ]:
mask = df_corr['price_aprox_usd'].isnull()
df_corr.drop(df_corr[mask].index, inplace=True)
df_corr.info()

Eliminamos aquellas filas que no tienen `surface_covered_in_m2` para que no distorcionen la correlacion.

In [ ]:
mask = df_corr['surface_covered_in_m2'].isnull()
df_corr.drop(df_corr[mask].index, inplace=True)
df_corr.info()

In [ ]:
df_corr